# CheXpert Dataset - Download and Data Preparation

This notebook shows how to download and preprocess the CheXpert dataset for further analysis with weakly supervised experiments. The CheXpert training set is composed of chest radiographs, which were annotated on the basis of reports using the rule-based CheXpert labeler. Each image is labeled with respect 12 pathologies as well as the observations "No Finding" and "Support Devices". For each of these categories, except "No Finding", the assigned weak label is either: (Irvin et al. (2019))

- positive (1.0)
- negative (0.0)
- not mentioned (blank)
- uncertain (-1.0) 

The development set was annotated by radiologists and therefore only contains the binary labels: (Irvin et al. (2019))

- positive (1.0) 
- negative (0.0)

You can register for obtaining the data under the following link: https://stanfordmlgroup.github.io/competitions/chexpert/. Once the registration is finished, you should receive an email which contains links for two different versions of the dataset, the original CheXpert dataset (around 439 GB) and a version with downsampled resolution (around 11 GB). The code below uses the downsampled version. Please unzip the downloaded folder in a directory of your choice and don't change the filenames or the folder structure, otherwise you might need to change some of the paths used in the following code in order for it to run properly. The zip file you obtained should contain a training and a validation set. The CheXpert test set is not publicly available, as it is used for the CheXpert competition (see link above). The reports that were used to label the images are also unavailable.

Please note that some output of the code below is not displayed due to the data use agreement on https://stanfordmlgroup.github.io/competitions/chexpert/.
Please run the code yourself with the data you downloaded from the above link.

The original CheXpert paper, "CheXpert: A large chest radiograph dataset with uncertainty labels and expert comparison" by Irvin et al. (2019), can be found here: https://arxiv.org/abs/1901.07031.

# Imports

First, let's import the required packages.

In [2]:
import os
from typing import List

import joblib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
import torch
import torchvision.transforms as transforms
from PIL import Image
from tabulate import tabulate
from torch.utils.data import Dataset, DataLoader
from torchvision.utils import save_image
from tqdm import tqdm

# Load the dataset

Now, let's load the files train.csv and valid.csv, which accompany the images. First, please change the working directory to the appropriate location by inserting the path to the folder in which you stored train.csv and valid.csv where "data_path" is mentioned in the code. If you didn't change the folder structure, this path should end with "\CheXpert-v1.0-small\CheXpert-v1.0-small".



In [3]:
path = "data_path"
os.chdir(path) # change working directory to appropriate location

## Get train data

In [4]:
training_set = pd.read_csv('train.csv')

Let's take a look at the first five rows of the raw training data. Please note that the small dataframe displayed below is entirely made up of fake entries. If you want to see the true data, please run training_set.head(5) yourself.

In [ ]:
training_set.head(5)

In [5]:
# dataframe with fake entries for demonstration purposes

fake_entries = list(zip(["CheXpert-v1.0-small/train/patient99999/study1/...", "CheXpert-v1.0-small/train/patient99999/study2/..."],
                        ["Female", "Female"], [44, 48], ["Frontal", "Frontal"], ["AP", "AP"], ["NaN", 1.0],
                        ["NaN", "NaN"], ["NaN", "NaN"], ["NaN", "NaN"], ["NaN", -1.0], [-1.0, "NaN"], ["NaN", "NaN"],
                        ["NaN", "NaN"], ["NaN", "NaN"], ["NaN", "NaN"], ["NaN", "NaN"], ["NaN", "NaN"], 
                        [1.0, -1.0], ["NaN", "NaN"]))

fake_training_set = pd.DataFrame(data=fake_entries, index=None, columns=training_set.columns, dtype=None, copy=None)

fake_training_set.head()

,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,CheXpert-v1.0-small/train/patient99999/study1/...,Female,44,Frontal,AP,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
1,CheXpert-v1.0-small/train/patient99999/study2/...,Female,48,Frontal,AP,1.0,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN


In [6]:
print("Number of observations in training set:", training_set.shape[0])

Number of observations in training set: 223414


As can be seen, each observation in the training set consists of a path to an image, some additional information about the patient and the nature of the image, as well as the weak labels for all 14 classes. 12 of the classes, "Enlarged Cardiomediastinum" to "Fracture", are considered pathologies. "No Finding" is assigned the label 1 (meaning "positive") if no pathology was marked as positive (1.0) or uncertain (-1.0) for this observation. Labels which were blank (meaning that the pathology was not mentioned in the report) turned into NaNs when loading the data. The training set has a total of 223414 observations. (Irvin et al. (2019))

## Get validation data

Now, let's do the same for the validation set.

In [9]:
validation_set = pd.read_csv("valid.csv")

In [ ]:
validation_set.head(5)

In [10]:
# dataframe with fake entries for demonstration purposes

fake_entries = list(zip(["CheXpert-v1.0-small/train/patient00000/study1/...", "CheXpert-v1.0-small/train/patient00001/study2/..."],
                        ["Male", "Female"], [65, 82], ["Frontal", "Lateral"], ["AP", "NaN"], [0.0, 0.0],
                        [0.0, 0.0], [0.0, 0.0], [1.0, 0.0], [0.0, 1.0], [1.0, 0.0], [0.0, 0.0], [0.0, 0.0],
                        [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 1.0]))

fake_validation_set = pd.DataFrame(data=fake_entries, index=None, columns=training_set.columns, dtype=None, copy=None)

fake_validation_set.head()

,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,CheXpert-v1.0-small/train/patient00000/study1/...,Male,65,Frontal,AP,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,CheXpert-v1.0-small/train/patient00001/study2/...,Female,82,Lateral,NaN,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [11]:
print("Number of observations in validation set:", validation_set.shape[0])

Number of observations in validation set: 234


As you can see, the validation set has the same structure as the training set. However, the validation set only uses positive (1.0) and negative (0.0) labels and no uncertainty (-1.0) or not-mentioned labels (blank). The validation set contains 234 observations. (Irvin et al. (2019))

## Collect statistics

In the following, some statistics are computed in order to get an idea about the label distribution in the two datasets.

In [12]:
training_labels = training_set.iloc[:, -13:-1]
labels_per_row = training_labels.count(axis = 1) # number of non-NaN labels per row in the training set

vals = pd.DataFrame(labels_per_row.value_counts())

# make a table
val_list = [(i, vals[0][i]) for i in vals.index]
    
print(tabulate(val_list, headers = ["Number of non-NaN labels", "Number of datapoints"]))

  Number of non-NaN labels    Number of datapoints
--------------------------  ----------------------
                         3                   64386
                         4                   50893
                         2                   50672
                         5                   23828
                         1                   23185
                         6                    7922
                         7                    1960
                         8                     294
                         0                     250
                         9                      19
                        10                       5


As can be seen, most training samples have at least a few pathologies, for which they have a label which is not blank, meaning it is either positive (1.0), negative (0.0) or uncertain (-1.0). Nevertheless, there seem to be 250 observations for which none of the 12 pathologies were mentioned in the corresponding report.

The following two tables should give an idea about the label distribution for the different pathologies in the training set and in the validation set.

In [13]:
val_list = []
for cond in training_labels.columns:
    vals = np.array(training_labels.value_counts(subset=[cond]).sort_index(ascending = True))
    val_list.append([cond, vals[0], vals[1], vals[2]])

print("Label distribution in the training set:", "\n")
print(tabulate(val_list, headers = ["Pathology", "-1.0", "0.0", "1.0"]))

Label distribution in the training set: 

Pathology                     -1.0    0.0     1.0
--------------------------  ------  -----  ------
Enlarged Cardiomediastinum   12403  21638   10798
Cardiomegaly                  8087  11116   27000
Lung Opacity                  5598   6599  105581
Lung Lesion                   1488   1270    9186
Edema                        12984  20726   52246
Consolidation                27742  28097   14783
Pneumonia                    18770   2799    6039
Atelectasis                  33739   1328   33376
Pneumothorax                  3145  56341   19448
Pleural Effusion             11628  35396   86187
Pleural Other                 2653    316    3523
Fracture                       642   2512    9040


In [14]:
validation_labels = validation_set.iloc[:, -13:-1]

val_list = []
for cond in validation_labels.columns:
    vals = np.array(validation_labels.value_counts(subset=[cond]).sort_index(ascending=True))
    if cond != "Fracture":
        val_list.append([cond, vals[0], vals[1]])
    else:
        val_list.append([cond, vals[0], 0]) # fracture never positively appears in validation set
        
print("Label distribution in the validation set:", "\n")
print(tabulate(val_list, headers = ["Pathology", "0.0", "1.0"]))

Label distribution in the validation set: 

Pathology                     0.0    1.0
--------------------------  -----  -----
Enlarged Cardiomediastinum    125    109
Cardiomegaly                  166     68
Lung Opacity                  108    126
Lung Lesion                   233      1
Edema                         189     45
Consolidation                 201     33
Pneumonia                     226      8
Atelectasis                   154     80
Pneumothorax                  226      8
Pleural Effusion              167     67
Pleural Other                 233      1
Fracture                      234      0


As can be seen, some observations like "Fracture" or "Lung Lesion", which are positively mentioned (i.e. label 1.0) for numerous observations in the training set, barely appear or don't appear at all in the validation set.

## Image preprocessing

Now that we have familiarized ourselves with the labels in the training and in the validation set, let's take a look at an example image from the training set. In order to do that, we first need to create paths that lead to the images.

In [ ]:
# paths to training images
image_paths_train = [os.path.join(path[: path.find("CheXpert-v1.0-small")], "CheXpert-v1.0-small", p) for p in training_set["Path"]]

# paths to validation images
image_paths_valid = [os.path.join(path[: path.find("CheXpert-v1.0-small")], "CheXpert-v1.0-small", p) for p in validation_set["Path"]]

sample_image = Image.open(image_paths_train[0]).convert('RGB')
plt.imshow(sample_image)
plt.show()
print("Dimensions of image:", sample_image.size)

All images in the (downsampled) dataset have a dimensionality of around 390 x 320 pixels, however, the individual image sizes vary. (Garbin et al. (2021))

In the following, a transform sequence is defined, which will be used to preprocess the images. The images are first resized to 224 x 224 pixels, since this is a requirement for many pre-trained CNNs in PyTorch, and then normalized with the mean and standard deviation from ImageNet. Such transformations have previously been applied by several other submissions regarding CheXpert on GitHub such as:
[this](https://github.com/gaetandi/cheXpert/blob/master/cheXpert_final.ipynb) and [this](https://github.com/Stomper10/CheXpert/blob/master/CheXpert_DenseNet121_FL.ipynb).

In [16]:
transform_list = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # normalization from ImageNet
    ])

The following class will be used to transform the images and prepare the dataset for PyTorch's DataLoader. Moreover, the uncertainty labels will be handled according to either the "U-Ones", "U-Zeroes", "U-Ignore" or "U-MultiClass" approach as they were described in the origianl CheXpert paper by Irvin et al. (2019).

You can specify how the uncertainty labels for each individual pathology will be handled by providing a list of the names of the pathologies to the arguments to_ones, to_zeros and to_ignore. to_ignore changes the uncertainty label (-1.0) to nan. Not specifying any pathology in any of the arguments to_ones, to_zeros or to_ignore will train the model with the "U-MultiClass" approach. 

The "Support Device" label as well as the "No Finding" label is dropped for all instances, since support devices do not count as pathologies and the "No Finding" label only depends on the labels for the remaining 12 pathologies. The "blank" labels, indicating that an observation was not mentioned in the corresponding report, will be treated as "uncertain".

If the return_image parameter is set to False, only the labels are returned. This might be useful if you already have the transformed images saved but wish to experiment with different settings of to_ones, to_zeros or to_ignore.

The validation set can be loaded by setting the subset input of the class to "valid" and supplying the appropriate image paths and number of images.

The structure of the class was inspired by the GitHub submissions credited above.

In [17]:
class CheXpertDatasetProcessor(Dataset):
    
    def __init__(self, 
                 path: str,
                 subset: str, 
                 image_paths: List[str], 
                 number_of_images: int,  
                 transform_sequence: list = None,
                 to_ones: List[str] = None,
                 to_zeros: List[str] = None, 
                 to_ignore: List[str] = None,
                 return_image: bool = True):
        
        """
        Args:
            path: path to the folder where train.csv and valid.csv are stored
            subset: either "train" to load the train.csv or "valid" to load valid.csv
            image_paths: paths to the images
            number_of_images: number of images in the dataset
            transform_sequence: sequence used to transform the images
            to_ones: list of pathologies for which uncertainty labels should be replaced by 1
            to_zeros: list of pathologies for which uncertainty labels should be replaced by 0
            to_ignore: list of pathologies for which uncertainty labels should be ignored (label will be turned to nan)
            return_image: True: image tensor and labels are returned, False: only labels are returned
        Returns: 
            224 x 224 image tensor and a corresponding tensor containing 12 labels
        """
        
        self.path = path
        self.subset = subset
        self.image_paths = image_paths
        self.number_of_images = number_of_images
        self.transform_sequence = transform_sequence
        self.to_ones = to_ones
        self.to_zeros = to_zeros
        self.to_ignore = to_ignore
        self.return_image = return_image
        
    def process_chexpert_dataset(self):
        
        # read dataset
        if self.subset == "train":
            data = pd.read_csv("train.csv")
            
        elif self.subset == "valid":
            data = pd.read_csv("valid.csv")
            
        else:
            raise ValueError("Invalid subset, please choose either 'train' or 'valid'")
            
        pathologies = data.iloc[:, -13:-1].columns
        
        # prepare labels
        data.iloc[:, -13:-1] = data.iloc[:, -13:-1].replace(float("nan"), -1) # blank labels -> uncertain
        
        if self.to_ones:
            if all(p in pathologies for p in self.to_ones): # check whether arguments are valid pathologies
                data[self.to_ones] = data[self.to_ones].replace(-1, 1) # replace uncertainty labels with ones
            else:
                raise ValueError("List supplied to to_ones contains invalid pathology, please choose from:",
                                 list(pathologies))
            
        if self.to_zeros:
            if all(p in pathologies for p in self.to_zeros):
                    data[self.to_zeros] = data[self.to_zeros].replace(-1, 0) # replace uncertainty labels with zeros
            else:
                raise ValueError("List supplied to to_zeros contains invalid pathology, please choose from:",
                                 list(pathologies))
            
        if self.to_ignore:
            if all(p in pathologies for p in self.to_ignore):
                    data[self.to_ignore] = data[self.to_ignore].replace(-1, float("nan")) # replace uncertainty labels with nan
            else:
                raise ValueError("List supplied to to_ignore contains invalid pathology, please choose from:",
                                     list(pathologies))
        
        self.data = data
    
    def __getitem__(self, index: int):
        
        """
        index: index of example that should be retrieved
        """
        
        if self.return_image not in [True, False]:
            raise ValueError("Please set return_image argument either to True or False")
        
        image_labels = self.data.iloc[index, -13:-1]
        
        if self.return_image is False: # only labels are returned, not the images
            return torch.tensor(image_labels)
        
        else:
            image_name = self.image_paths[index]
        
            patient_image = Image.open(image_name).convert('RGB')
        
            if self.transform_sequence:
                patient_image = self.transform_sequence(patient_image) # apply the transform_sequence if one is specified
        
            else:
                # even if no other transformation is applied, the image should be turned into a tensor
                to_tensor = transforms.ToTensor()
                patient_image = to_tensor(patient_image)
            
            return patient_image, torch.tensor(image_labels)
    
    def __len__(self):
        return self.number_of_images

In [18]:
# prepare training data
chexpert_train = CheXpertDatasetProcessor(path = path, subset = "train", image_paths = image_paths_train,
                                          number_of_images = training_set.shape[0], transform_sequence = transform_list)
chexpert_train.process_chexpert_dataset()

# prepare validation data
chexpert_valid = CheXpertDatasetProcessor(path = path, subset = "valid", image_paths = image_paths_valid,
                                          number_of_images = validation_set.shape[0], transform_sequence = transform_list)
chexpert_valid.process_chexpert_dataset()

For a given index, the getitem function returns the tensor representing the preprocessed image as well as a tensor containing all of the labels for this observation.

Let's take a look at what getitem returns for the training example with index 0.

In [ ]:
chexpert_train.__getitem__(0)

In [20]:
shape_of_image_tensor = chexpert_train.__getitem__(0)[0].shape
shape_of_label_tensor = chexpert_train.__getitem__(0)[1].shape

print("Shape of image tensor:", shape_of_image_tensor)
print("Shape of label tensor:", shape_of_label_tensor)

Shape of image tensor: torch.Size([3, 224, 224])
Shape of label tensor: torch.Size([12])


## Store the preprocessed data

The tensors representing the preprocessed images and the tensors containing the labels can be stored on your computer using joblib. If you want to do this, you can run the following code but please be aware that the resulting joblib files might become very large, especially for the training data.

In [ ]:
filename_train = 'chexpert_data_train_labels.joblib'
outfile = open(filename_train,'wb')

for i in tqdm(range(0, training_set.shape[0])):
    joblib.dump(chexpert_train.__getitem__(i)[1], outfile)
    
filename_valid = 'chexpert_data_valid_labels.joblib'
outfile = open(filename_valid,'wb')

for i in tqdm(range(0, validation_set.shape[0])):
    joblib.dump(chexpert_valid.__getitem__(i)[1], outfile)

## Finish

This concludes the preprocessing of the CheXpert data.

### References

CheXpert: A large chest radiograph dataset with uncertainty labels and expert comparison by Irvin et al. (2019):
https://arxiv.org/abs/1901.07031

Structured dataset documentation: a datasheet for CheXpert by Garbin et al. (2021):
https://arxiv.org/pdf/2105.03020.pdf